In [ ]:
#Imports
import torch
import matplotlib.pyplot as plt
import csv
import cProfile
import pstats

from parameter_functions import distance_mse, distance_null, distance_discrete, get_prisoners_dilemma, get_stag_hunt, get_uniform_game, get_normal_game, Net
from evidential_cooperation import Agent, Environment, get_average_scores
from agent_generation import generate_twins, diverging_twins

In [ ]:
# Get average scores over multiple runs with agents trained on evidential cooperation
distance_f, distance_f_name = distance_mse, 'MSE'
game_generator, game_generator_name = get_prisoners_dilemma, 'PD'
agents_generator, agents_generator_name = diverging_twins, 'DIVERGING'
num_rounds, num_games, num_trials = 3000, 1, 1 # runtime is proportional to num_rounds * num_games * num_trials
death_rate = 0.0
net_n, agents_per_net = 2, 2 # number agents = net_n * agents_per_net
random_inputs = 0
lr = 0.001

average_scores, nets = None, None
cProfile.run('average_scores, nets = get_average_scores(agents_generator=lambda: agents_generator(distance_f=distance_f, net_n=net_n, agents_per_net=agents_per_net, random_inputs=random_inputs, lr=lr), game_generator=game_generator, death_rate=death_rate, num_trials=num_trials, num_rounds=num_rounds, num_games=num_games, random_inputs=random_inputs, verbose=True)', 'stats/stats {}nets {}agents {} {} {} {}rounds {}games {}death {}trials'.format(net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, num_trials))

# Plot and save the average scores
plt.clf()
plt.plot(average_scores)
plt.savefig('graphs/{}nets {}agents {} {} {} {}rounds {}games {}death {}trials.png'.format(net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, num_trials))
plt.show()
with open('graphs/{}nets {}agents {} {} {} {}rounds {}games {}death {}trials.csv'.format(net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, num_trials), 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(average_scores)

# Save the trained networks
for i, net in enumerate(nets):
    torch.save(net.state_dict(), 'models/net{} {}nets {}agents {} {} {} {}rounds {}games {}death {}trials'.format(i, net_n, net_n*agents_per_net, game_generator_name, distance_f_name, agents_generator_name, num_rounds, num_games, death_rate, num_trials))

print('Average score: {}'.format(sum(average_scores)/num_rounds))

In [ ]:
distance_f, distance_f_name = distance_mse, 'MSE'
game_generator, game_generator_name = get_prisoners_dilemma, 'PD'
num_rounds, num_games, death_rate, num_trials = 1000, 1, 0.0, 100
net_n = 2
agents_per_net = 2

nets = [Net(hidden_size=16) for _ in range(net_n)]
[net.load_state_dict(torch.load('models/net{}|{} {} {} {}rounds {}games {}death.pt'.format(i+1, net_n, game_generator_name, distance_f_name, num_rounds, num_games, death_rate))) for i, net in enumerate(nets)]


print("Auto-cooperation: ", end='')
for net in nets:
    print(net(torch.cat((get_prisoners_dilemma().flatten(), torch.tensor([0])))), end=' ')
print("\nCooperation: ", end='')
for net in nets:
    print(net(torch.cat((get_prisoners_dilemma().flatten(), torch.tensor([distance_mse(nets[0], nets[1])])))), end=' ')

In [ ]:
p = pstats.Stats('stats/stats 2nets 4agents PD MSE DIVERGING 3000rounds 1games 0.0death 1trials')
p.strip_dirs().sort_stats('cumulative').print_stats()